In [1]:
import pandas as pd
import numpy as np


####solver
from scipy.optimize import fsolve


import matplotlib.pyplot as plt
%matplotlib inline


## Short Interest Tree Model

### Example from book "Fixed Income Securities by Pietro Veronesi

I am using this book as a reference to create my functions then to test them. 
At a later stage, I will try to calibrate "theta".

### Ho-Lee and Black-Derman-Toy model.

In [3]:
class Tree(object):
    
    '''
    This class will plot several tree (Forward tree).
    ============================
    n: number of time period
    T: number of years. 
    dt: T/n
    r0: initial interest rate
    sigma = std / volatility
    
    =============================
    fake_tree: Just draw a tree.
    Ho Lee:
    Black Derman Toy: 
    
    =============================
    '''
    
    def __init__(self,T,n, r0, sigma):
        
        self.n = n
        self.T = T
        self.r0 = r0 # initial interest rate
        self.sigma = sigma # Annualised volatility 
        self.dt = T/n


    def tree_bond(self,rate):
        '''
        backward tree for Bond
        For check purposes only.
        '''
        bond = np.zeros((self.n+1,self.n+1)) ##bond price tree
        bond[:,-1] = 100

        for i in range(self.n-1,-1,-1):
            for j in range(self.n-1,-1,-1):
                if j>i:
                    continue
                bond[j,i] = np.exp(-rate[j,i] * self.dt)*0.5 *(bond[j,i+1] +bond[j+1,i+1])

        return bond[0,0]
        
    @staticmethod
    def r2b(rate):
        '''
        rate to bond (zero coupon bond)
        '''
        return -self.dt * np.log(rate)
    
    def ho_lee_improve(self,theta):
        
        """
        In progress
        testing new implementation
        """
        if theta ==0:
            theta = np.zeros((self.n+1))
        
        tree = np.zeros((n+1,n+1))
        
        for i in range(self.n+1):
            tree[:i+1,i:i+1] = (tree[:i] * self.u**(np.arange(i,-1,-1)) \
                * self.d**(np.arange(0,i+1,1))).reshape(-1,1)
            
            tree[0,i+1] = tree[0,i] +theta[i] * self.dt + sigma * np.sqrt(self.dt)

    def ho_lee(self,r0,sigma, theta):
        
        '''
        In financial mathematics, the Ho–Lee model is a short rate model widely used 
        in the pricing of bond options, swaptions and other interest rate derivatives,
        and in modeling future interest rates. It was developed in 1986 by Thomas Ho and Sang Bin Lee.
        (from Wikepedia: https://en.wikipedia.org/wiki/Ho%E2%80%93Lee_model)
        '''
        
        if theta == 0:
            theta = np.zeros((self.n+1))

        
        p = 0.5
        
        tree = np.zeros((self.n+1,self.n+1))
        tree[0,0] = r0

        #print the forward tree 
        for i in range(len(tree)-1):
            
            tree[0,i+1] = tree[0,i] +theta[i] * self.dt + sigma * np.sqrt(self.dt)
            

            for j in range(len(tree)-1):
                if j >i:
                    next
                else:
                    tree[j+1,i+1] = tree[j,i] +theta[i] * self.dt - sigma * np.sqrt(self.dt)
        
        return tree

    def bdt(self,r0,sigma, theta):
        '''
        In mathematical finance, the Black–Derman–Toy model (BDT) is a popular short rate model
        used in the pricing of bond options, swaptions and other interest rate derivatives.
        Wikipedia: https://en.wikipedia.org/wiki/Black%E2%80%93Derman%E2%80%93Toy_model
        '''
        
        z0 = np.log(r0)
        
        if theta == 0:
            theta = np.zeros((self.n+1))

        p = 0.5
        
        tree = np.zeros((self.n+1,self.n+1))
        tree[0,0] = z0

        #print the forward tree 
        for i in range(len(tree)-1):
            
            tree[0,i+1] = tree[0,i] +theta[i] * self.dt + sigma * np.sqrt(self.dt)
            

            for j in range(len(tree)-1):
                if j >i:
                    next
                else:
                    tree[j+1,i+1] = tree[j,i] +theta[i] * self.dt - sigma * np.sqrt(self.dt)
        
        
        # z_i = ln(r_i) <=> r_i = exp(z_i)
        tree1 = np.exp(tree)
        
        #exp(0) = 1 ==> replace 1 by 0
        for i in range(len(tree1)):    
            for j in range(len(tree1)):
                if j >i:
                    tree1[j,i] = 0
        
        
        return tree1


In [66]:
# Table 10.11 Zero Coupon Bond Prices on January 8, 2002,
# Source The Wall Street Journal
# Taken from Fixed Income Securities - Pietro Veronesi || p.370


maturity = np.arange(0.5,6.0,0.5)

price = [99.1338, 97.8925, 96.1462,
         94.1011, 91.7136, 89.2258,
         86.8142, 8405016, 82.1848,
         79.7718, 77.4339]

yield_ = [1.74, 2.13, 2.62, 3.04,
          3.46,3.8,4.04,4.21,4.36, 
          4.52,4.65]

df = pd.DataFrame([maturity, price, yield_]).T
df.columns = ["Maturity", "Price", "Yield"]

In [67]:
df

,Maturity,Price,Yield
0,0.5,9.913380e+01,1.74
1,1.0,9.789250e+01,2.13
2,1.5,9.614620e+01,2.62
3,2.0,9.410110e+01,3.04
4,2.5,9.171360e+01,3.46
5,3.0,8.922580e+01,3.80
6,3.5,8.681420e+01,4.04
7,4.0,8.405016e+06,4.21
8,4.5,8.218480e+01,4.36
9,5.0,7.977180e+01,4.52


In [52]:
test = np.zeros((4,4))
test[0,0] = 0.5

In [57]:
test[0,1:] = test[0,0] + 0.2 * np.arange(1,4,1) * 0.25

for i in range(1,len(test)):
    test[i,i:] = test[0,0] - 0.2 * np.arange(i,4,1) * 0.25

In [58]:
test[:,:]

array([[0.5 , 0.55, 0.6 , 0.65],
       [0.  , 0.45, 0.4 , 0.35],
       [0.  , 0.  , 0.4 , 0.35],
       [0.  , 0.  , 0.  , 0.35]])

In [14]:
test[0,0] = 100

#print the forward tree 
for i in range(len(tree)-1):
    
    tree[0,i+1] = tree[0,i] +theta[i] * self.dt + sigma * np.sqrt(self.dt)
    

    for j in range(len(tree)-1):
        if j >i:
            next
        else:
            tree[j+1,i+1] = tree[j,i] +theta[i] * self.dt - sigma * np.sqrt(self.dt)


array([4, 3, 2, 1])

##### Table: 11.2; p 384

In [4]:
t = Tree(1.5,3)
t.v = t.ho_lee(0.0174,0.0173, theta)
t.tree_bond(t.v)


NameError: name 'theta' is not defined

In [ ]:
pd.DataFrame(Tree(1.5,3).tree_bond(t[:3,:3]))

96.14627330741644

In [ ]:
theta = [0.015674,0.021824,0.014374,0.017324,0.007873,0.000423,-0.000628,0.004322,0.009271,0.001202]
pd.DataFrame(Tree(5,10).ho_lee(0.0174,0.0173, theta))

,0,1,2,3,4,5,6,7,8,9,10
0,0.0174,0.037470,0.060615,0.080035,0.100930,0.117099,0.129544,0.141463,0.155857,0.172725,0.185559
1,0.0000,0.013004,0.036149,0.055569,0.076464,0.092633,0.105078,0.116997,0.131391,0.148259,0.161093
2,0.0000,0.000000,0.011683,0.031103,0.051998,0.068167,0.080612,0.092531,0.106925,0.123793,0.136627
3,0.0000,0.000000,0.000000,0.006637,0.027532,0.043702,0.056146,0.068065,0.082459,0.099327,0.112161
4,0.0000,0.000000,0.000000,0.000000,0.003066,0.019236,0.031680,0.043599,0.057993,0.074861,0.087695
5,0.0000,0.000000,0.000000,0.000000,0.000000,-0.005230,0.007214,0.019133,0.033527,0.050396,0.063230
6,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.017252,-0.005333,0.009061,0.025930,0.038764
7,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.029799,-0.015405,0.001464,0.014298
8,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.039871,-0.023002,-0.010168
9,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.047468,-0.034634


##### Black Derman & Toy
Table 11.3; p385

In [ ]:
theta1 = [0.7182,0.6916,0.3348,0.3379,0.1182,-0.023,-0.0438,0.0455,0.1281,-0.0126]
pd.DataFrame(Tree(5,10).bdt(0.0174,0.2142,theta1))

,0,1,2,3,4,5,6,7,8,9,10
0,0.0174,0.028992,0.047670,0.065573,0.090339,0.111512,0.128264,0.146007,0.173794,0.215590,0.249272
1,0.0000,0.021415,0.035211,0.048435,0.066729,0.082369,0.094743,0.107849,0.128373,0.159247,0.184126
2,0.0000,0.000000,0.026009,0.035777,0.049290,0.060842,0.069982,0.079663,0.094824,0.117629,0.136005
3,0.0000,0.000000,0.000000,0.026427,0.036408,0.044941,0.051693,0.058844,0.070042,0.086887,0.100461
4,0.0000,0.000000,0.000000,0.000000,0.026893,0.033196,0.038183,0.043465,0.051737,0.064179,0.074206
5,0.0000,0.000000,0.000000,0.000000,0.000000,0.024520,0.028204,0.032106,0.038216,0.047406,0.054813
6,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.023715,0.028228,0.035017,0.040488
7,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017517,0.020851,0.025865,0.029906
8,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015402,0.019106,0.022090
9,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014112,0.016317


### Caps and Floor

Caps are like european calls <br>
Floors are like european puts

In [ ]:
class Option_IR(object):
    
    def __init__(self,T, n):
        ''' in case I need to write something'''
        
        self.T = T        
        self.n = n

        self.dt = T/n
       
    
    def ctns_rate(self,r):
        
        return (np.exp(r*self.dt)-1)/self.dt
        
        
        
    def european(self, tree_rate, Strike,N, cp = "cap"):



        p = 0.5
        
        tree_ctns = self.ctns_rate(tree_rate)
        
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        
        for i in range(self.n+1):
            for j in range(self.n+1):
                if j>i:
                    tree_cf[j,i] = 0
                else:
                    if cp =='cap':
                        tree_cf[j,i] = self.dt * N * max(tree_ctns[j,i] - Strike,0)
                    else:
                        tree_cf[j,i] = self.dt * N * max(Strike - tree_ctns[j,i],0)
                    
        tree_eu = np.zeros((self.n+1,self.n+1))
        
        
        #intresic value
        for j in range(self.n+1):
            tree_eu[j,-1] = np.exp(-tree_rate[j,-1] * self.dt) * tree_cf[j,-1]
            
            
                

        #Backward tree (European option)
        for i in range(self.n,0,-1):
            for j in range(i):
                tree_eu[j,i-1] = np.exp(-tree_rate[j,i-1] * self.dt)*(( p * tree_eu[j,i] + (1-p) * tree_eu[j+1,i])+tree_cf[j,i-1])

        return tree_cf, tree_eu
    

    
    def swap_rate(self, zcb):
        
        return 1/self.dt * (1 - zcb[-1])/ (sum(zcb))
    

    def swap(self, tree_rate, c, N):
        
        p = 0.5
        
        tree_ctns = self.ctns_rate(tree_rate)
        
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        
        for i in range(self.n+1):
            for j in range(self.n+1):
                if j>i:
                    tree_cf[j,i] = 0
                else:
                    tree_cf[j,i] = self.dt * N * (tree_ctns[j,i] - c)
                    
        tree_eu = np.zeros((self.n+1,self.n+1))
        
        
        #intresic value
        for j in range(self.n+1):
            tree_eu[j,-1] = np.exp(-tree_rate[j,-1] * self.dt) * tree_cf[j,-1]
            
            
                

        #Backward tree (European option)
        for i in range(self.n,0,-1):
            for j in range(i):
                tree_eu[j,i-1] = np.exp(-tree_rate[j,i-1] * self.dt)*(( p * tree_eu[j,i] + (1-p) * tree_eu[j+1,i])+tree_cf[j,i-1])

        return tree_cf, tree_eu


In [ ]:
pd.DataFrame(Option_IR(4.5,9).swap(y,c, 100)[1])

,0,1,2,3,4,5,6,7,8,9
0,0.00689,4.272425,8.188949,11.385667,13.866054,15.227770,15.502382,14.717643,12.586277,8.204988
1,0.00000,-1.519939,2.046496,5.049234,7.484485,9.030847,9.717720,9.580177,8.417565,5.582189
2,0.00000,0.000000,-2.785965,0.050314,2.445034,4.146658,5.180691,5.581978,5.209567,3.596844
3,0.00000,0.000000,0.000000,-3.826893,-1.466314,0.361367,1.677267,2.512431,2.767122,2.103602
4,0.00000,0.000000,0.000000,0.000000,-4.463328,-2.535859,-0.996939,0.179442,0.922178,0.985778
5,0.00000,0.000000,0.000000,0.000000,0.000000,-4.732939,-3.020801,-1.580577,-0.463315,0.151905
6,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.542809,-2.901052,-1.499293,-0.468546
7,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.887730,-2.271460,-0.929320
8,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.845639,-1.271028
9,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.524176


##### Assign your tree to a variable x:

In [ ]:
x = Tree(1,2).bdt(0.0174,0.2142,theta1)
pd.DataFrame(x)

,0,1,2
0,0.0174,0.028992,0.047670
1,0.0000,0.021415,0.035211
2,0.0000,0.000000,0.026009


##### Cash Flow tree: (p.390)

In [ ]:
pd.DataFrame(Option_IR(1,2).european(x,0.025,100,'cap')[0]) # Cash flow( Please note it arrives a T+1)

,0,1,2
0,0.0,0.210176,1.162114
1,0.0,0.000000,0.526160
2,0.0,0.000000,0.058947


##### Cap Value Tree: (p.391)

In [ ]:
pd.DataFrame(Option_IR(1,2).european(x,0.025,100,'cap')[1])

,0,1,2
0,0.647167,1.021126,1.134743
1,0.000000,0.284518,0.516978
2,0.000000,0.000000,0.058185


##### Cash Flow, 5-year Cap: Panel A, p 392

In [ ]:
y = Tree(4.5,9).bdt(0.0174,0.2142,theta1)
pd.DataFrame(Option_IR(4.5,9).european(y,0.025,100,'cap')[0])

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.210176,1.162114,2.082966,3.370518,4.483962,5.373335,6.323442,7.828410,10.131965
1,0.0,0.000000,0.526160,1.201337,2.142751,2.954425,3.601152,4.290489,5.379150,7.037926
2,0.0,0.000000,0.058947,0.554951,1.245117,1.838851,2.311063,2.813549,3.605376,4.807823
3,0.0,0.000000,0.000000,0.080115,0.587084,1.022503,1.368336,1.735888,2.314145,3.190092
4,0.0,0.000000,0.000000,0.000000,0.103738,0.423658,0.677501,0.947041,1.370591,2.011013
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.170201,0.368239,0.679156,1.148637
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.171417,0.516267
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051672
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Cash Flow, 5-year Cap: Panel B, p 392

In [ ]:
pd.DataFrame(Option_IR(4.5,9).european(y,0.025,100,'cap')[1])

,0,1,2,3,4,5,6,7,8,9
0,9.439097,12.18811,15.098181,17.348980,18.926699,19.437904,18.889858,17.286189,14.325789,9.096594
1,0.000000,6.85504,9.213622,11.251525,12.761801,13.423482,13.247149,12.249134,10.218022,6.499271
2,0.000000,0.00000,4.644049,6.450698,7.890201,8.680570,8.819725,8.328137,7.056592,4.533210
3,0.000000,0.00000,0.000000,2.841083,4.134157,5.003337,5.399971,5.317338,4.649420,3.054471
4,0.000000,0.00000,0.000000,0.000000,1.463359,2.242706,2.789097,3.028711,2.831020,1.947505
5,0.000000,0.00000,0.000000,0.000000,0.000000,0.516157,0.924069,1.302001,1.465404,1.121731
6,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.120979,0.231983,0.444260,0.507306
7,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012510,0.025240,0.051008
8,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###### Zero Coupon Bond Price on January 8, 2002 (p 370)

In [ ]:
zcb_08 = [99.1338, 97.8925,
      96.462, 94.1011,
      91.7136, 89.2258,
      86.8142, 84.5016,
      82.1848, 79.7718, 77.4339]

In [ ]:
del zcb_08[-1] # remove the last element of the list
zcb = [i/100 for i in zcb_08] # divide each element by 100
zcb


[0.9913379999999999,
 0.9789249999999999,
 0.96462,
 0.941011,
 0.917136,
 0.8922580000000001,
 0.868142,
 0.845016,
 0.8218479999999999,
 0.797718]

##### Calculating c as per example 11.5 page 393. 
please note that the formula should be 2x (and not 1/2!)

In [ ]:
c = Option_IR(1,2).swap_rate(zcb)
c

0.04486177219546836